In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE89594"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE89594"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE89594.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE89594.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE89594.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrated network analysis reveals genotype-phenotype correlations in Williams syndrome"
!Series_summary	"Williams Syndrome (WS) is a rare neurodevelopmental disorder caused by heterozygous deletions in a chromosome 7q11.23 region typically encompassing 26-28 genes. WS patients exhibit a wide spectrum of symptoms, including cardiovascular disease, intellectual disability, visuospatial deficits and hypersociability a behavioral profile that contrasts with autism spectrum disorder (ASD). However, the relationship between neuropsychiatric phenotypes and dysregulated gene networks caused by the 7q11.23 deletion is unknown. We report results from a large-scale integrated transcriptome analysis of peripheral blood in clinically evaluated subjects with WS, ASD and matched controls. We identified significantly differential expressed genes in WS as compared with ASD or controls, even after removing genes spanning the 7q11.23 region. Using weighted gene co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The background information suggests this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows for each variable in the sample characteristics

# For trait (Intellectual Disability):
# The diagnosis row (0) contains Williams Syndrome, ASD, and control status
# Williams Syndrome is associated with intellectual disability according to the background info
trait_row = 0

# For age:
# Age information is available in row 2
age_row = 2

# For gender:
# Gender information is available in row 3
gender_row = 3

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert diagnosis to intellectual disability status
    Williams Syndrome (WS) has intellectual disability = 1
    Control and ASD are set to 0
    """
    if not value or ':' not in value:
        return None
    
    diagnosis = value.split(':', 1)[1].strip().lower()
    
    if 'williams syndrome' in diagnosis or 'ws' in diagnosis:
        return 1  # Williams Syndrome patients have intellectual disability
    elif 'control' in diagnosis or 'asd' in diagnosis or 'autism' in diagnosis:
        return 0  # Controls and ASD patients are the reference group
    else:
        return None  # Unknown values

def convert_age(value):
    """Convert age string to numeric value in years"""
    if not value or ':' not in value:
        return None
    
    try:
        # Extract the age value, typically in format "age: XXy"
        age_str = value.split(':', 1)[1].strip()
        # Remove 'y' and convert to integer
        if 'y' in age_str:
            age = int(age_str.replace('y', '').strip())
            return age
        else:
            return None
    except:
        return None

def convert_gender(value):
    """Convert gender to binary: female=0, male=1"""
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().lower()
    
    if 'female' in gender:
        return 0
    elif 'male' in gender:
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM2384988': [0.0, 22.0, 0.0], 'GSM2384989': [0.0, 23.0, 0.0], 'GSM2384990': [0.0, 24.0, 0.0], 'GSM2384991': [0.0, 24.0, 0.0], 'GSM2384992': [0.0, 33.0, 1.0], 'GSM2384993': [0.0, 22.0, 1.0], 'GSM2384994': [0.0, 24.0, 0.0], 'GSM2384995': [0.0, 21.0, 1.0], 'GSM2384996': [0.0, 24.0, 1.0], 'GSM2384997': [0.0, 20.0, 0.0], 'GSM2384998': [0.0, 28.0, 0.0], 'GSM2384999': [0.0, 21.0, 1.0], 'GSM2385000': [0.0, 21.0, 0.0], 'GSM2385001': [0.0, 22.0, 1.0], 'GSM2385002': [0.0, 25.0, 1.0], 'GSM2385003': [0.0, 23.0, 0.0], 'GSM2385004': [0.0, 20.0, 0.0], 'GSM2385005': [0.0, 21.0, 1.0], 'GSM2385006': [0.0, 20.0, 0.0], 'GSM2385007': [0.0, 32.0, 1.0], 'GSM2385008': [0.0, 36.0, 0.0], 'GSM2385009': [0.0, 24.0, 1.0], 'GSM2385010': [0.0, 21.0, 1.0], 'GSM2385011': [0.0, 30.0, 0.0], 'GSM2385012': [0.0, 28.0, 1.0], 'GSM2385013': [0.0, 22.0, 1.0], 'GSM2385014': [0.0, 24.0, 0.0], 'GSM2385015': [0.0, 21.0, 1.0], 'GSM2385016': [0.0, 22.0, 1.0], 'GSM2385017': [0.0, 20.0, 0.0], 'GSM2385018': [0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 62976 rows
First 20 gene IDs:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The extracted gene IDs appear to be numeric identifiers (1, 2, 3, etc.)
# These are not standard human gene symbols, which would typically be alphanumeric 
# identifiers like BRCA1, TP53, etc.
# Therefore, these identifiers need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5982814 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC01627

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the correct columns for mapping
# From the annotation preview, we can see:
# 'ID' in gene annotation corresponds to gene identifiers in the expression data
# 'GENE_SYMBOL' contains the gene symbols we want to map to

# 2. Getting the gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print(f"Created mapping dataframe with {len(mapping_df)} rows")

# Preview the mapping
print("\nMapping preview (first few rows):")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level data to gene expression data
print("\nApplying gene mapping to convert probe data to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data has {len(gene_data)} genes and {gene_data.shape[1]} samples")

# Preview the gene expression data
print("\nGene expression data preview (first few genes):")
print(preview_df(gene_data))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping dataframe...


Created mapping dataframe with 54295 rows

Mapping preview (first few rows):
{'ID': ['4', '5', '6', '7', '8'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}

Applying gene mapping to convert probe data to gene expression data...
Converted gene expression data has 20353 genes and 94 samples

Gene expression data preview (first few genes):
{'GSM2384988': [14.514778639, 7.770994141, 50.618383177, 15.846223135999999, 54.533775519], 'GSM2384989': [16.325146179, 8.384069535, 50.382498582, 13.472051642, 52.761437181], 'GSM2384990': [15.077915928, 8.309437624, 53.497208234, 12.255831167, 53.993246103], 'GSM2384991': [16.024857048999998, 8.676199029, 52.469042418, 13.262691837, 53.174300299], 'GSM2384992': [14.205910164, 7.552051737, 54.605664255, 12.584846452, 54.944290563], 'GSM2384993': [15.044565859999999, 8.225743264, 52.634522866, 15.048242242, 56.156791729], 'GSM2384994': [15.585593495000001, 8.360763022, 53.430031596999996, 14.791714749, 54.135796581], 'GSM2384995': [15.

Gene expression data saved to ../../output/preprocess/Intellectual_Disability/gene_data/GSE89594.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the expression data...")
try:
    # If previous steps have already loaded gene_data
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data from file and link with genetic data
print("\nLoading clinical data from file...")
try:
    # Load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
    
    # Set is_trait_available based on whether the clinical data contains the trait
    is_trait_available = True
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    print("\nLinking clinical and genetic data...")
    # Format clinical data for linking - transpose it so samples are rows
    clinical_df_t = selected_clinical_df.T
    clinical_df_t.columns = [trait, 'Age', 'Gender']
    
    # Link clinical and genetic data
    linked_data = pd.merge(clinical_df_t, normalized_gene_data.T, 
                          left_index=True, right_index=True)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 3. Handle missing values systematically
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After handling missing values, data shape: {linked_data.shape}")
    
    # 4. Determine whether features are biased
    print("\nChecking for bias in features...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Final validation and save metadata
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased,
        df=linked_data,
        note="Williams Syndrome patients as intellectual disability cases, with ASD and controls as reference group."
    )
    
    # 6. Save the linked data if usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        
        # Save linked data
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Dataset not usable for {trait} association studies. Data not saved.")

except Exception as e:
    print(f"Error in data linking or processing: {e}")
    # Create a minimal dataframe for validation purposes
    linked_data = pd.DataFrame({trait: [0, 1]})
    
    # Perform final validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available, 
        is_trait_available=is_trait_available,
        is_biased=True,  # Not relevant since data isn't usable
        df=linked_data,
        note="Failed to link gene and clinical data: " + str(e)
    )
    print(f"Dataset usability: {is_usable}")

Normalizing gene symbols in the expression data...


Normalized gene data saved to ../../output/preprocess/Intellectual_Disability/gene_data/GSE89594.csv
Normalized gene data shape: (19847, 94)

Loading clinical data from file...
Clinical data shape: (3, 94)

Linking clinical and genetic data...
Linked data shape: (94, 19850)

Handling missing values...


After handling missing values, data shape: (94, 19850)

Checking for bias in features...
For the feature 'Intellectual_Disability', the least common label is '1.0' with 32 occurrences. This represents 34.04% of the dataset.
The distribution of the feature 'Intellectual_Disability' in this dataset is fine.

Quartiles for 'Age':
  25%: 20.0
  50% (Median): 22.5
  75%: 27.0
Min: 10.0
Max: 43.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 47 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Intellectual_Disability/GSE89594.csv
